In [10]:
from Deepfill import utils
import argparse
import cv2
import torch
import numpy as np
# parser = argparse.ArgumentParser()
# # General parameters
# parser.add_argument('--results_path', type = str, default = './results', help = 'testing samples path that is a folder')
# parser.add_argument('--gan_type', type = str, default = 'WGAN', help = 'the type of GAN for training')
# parser.add_argument('--gpu_ids', type = str, default = "0", help = 'gpu ids: e.g. 0  0,1,2, 0,2. use -1 for CPU')
# parser.add_argument('--cudnn_benchmark', type = bool, default = True, help = 'True for unchanged input data type')
# # Training parameters
# parser.add_argument('--epoch', type = int, default = 40, help = 'number of epochs of training')
# parser.add_argument('--batch_size', type = int, default = 1, help = 'size of the batches')
# parser.add_argument('--num_workers', type = int, default = 8, help = 'number of cpu threads to use during batch generation')
# # Network parameters
# parser.add_argument('--in_channels', type = int, default = 4, help = 'input RGB image + 1 channel mask')
# parser.add_argument('--out_channels', type = int, default = 3, help = 'output RGB image')
# parser.add_argument('--latent_channels', type = int, default = 48, help = 'latent channels')
# parser.add_argument('--pad_type', type = str, default = 'zero', help = 'the padding type')
# parser.add_argument('--activation', type = str, default = 'lrelu', help = 'the activation type')
# parser.add_argument('--norm', type = str, default = 'in', help = 'normalization type')
# parser.add_argument('--init_type', type = str, default = 'xavier', help = 'the initialization type')
# parser.add_argument('--init_gain', type = float, default = 0.02, help = 'the initialization gain')

# opt = parser.parse_args()

class Config:
    def __init__(self):
        self.in_channels = 4
        self.out_channels = 3
        self.latent_channels=48
        self.pad_type = 'zero'
        self.activation = 'elu'
        self.norm='none'
        self.init_type = 'xavier'
        self.init_gain=0.02

opt = Config()

generator = utils.create_generator(opt)
generator.load_state_dict(torch.load('Deepfill\\deepfillv2_WGAN_G_epoch40_batchsize4.pth'))
generator.eval()
generator.cuda()


img = cv2.imread('data\\image1.jpg')
mask = cv2.imread('data\\mask1.jpg')[:, :, 0]
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img = torch.from_numpy(img.astype(np.float32) / 255.0).permute(2, 0, 1).contiguous()
mask = torch.from_numpy(mask.astype(np.float32) / 255.0).unsqueeze(0).contiguous()



img = torch.stack([img])
mask = torch.stack([mask])

img = img.cuda()
mask = mask.cuda()

with torch.no_grad():
    first_out, second_out = generator(img, mask)

# forward propagation
first_out_wholeimg = img * (1 - mask) + first_out * mask        # in range [0, 1]
second_out_wholeimg = img * (1 - mask) + second_out * mask     # in range [0, 1]

# print(img)

masked_img = img * (1 - mask) + mask
mask = torch.cat((mask, mask, mask), 1)
img_list = [second_out_wholeimg]
name_list = ['second_out']
utils.save_sample_png(sample_folder = "", sample_name = '%d' % (0+ 1), img_list = img_list, name_list = name_list, pixel_max_cnt = 255)

Generator is created!
Initialize generator with xavier type
